# ATTN: This script uses Google translate to detect job description language. Google translate will limit requests and take a very long time. Only run this script if redoing language detection.

# Read from scraped data

In [1]:
import os # type:ignore # isort:skip # fmt:skip # noqa # nopep8
import sys # type:ignore # isort:skip # fmt:skip # noqa # nopep8
from pathlib import Path # type:ignore # isort:skip # fmt:skip # noqa # nopep8

mod = sys.modules[__name__]

code_dir = None
code_dir_name = 'Code'
unwanted_subdir_name = 'Analysis'

if code_dir_name not in str(Path.cwd()).split('/')[-1]:
    for _ in range(5):

        parent_path = str(Path.cwd().parents[_]).split('/')[-1]

        if (code_dir_name in parent_path) and (unwanted_subdir_name not in parent_path):

            code_dir = str(Path.cwd().parents[_])

            if code_dir is not None:
                break
else:
    code_dir = str(Path.cwd())
sys.path.append(code_dir)

# %load_ext autoreload
# %autoreload 2


In [2]:
from setup_module.imports import * # type:ignore # isort:skip # fmt:skip # noqa # nopep8


0it [00:00, ?it/s]

In [3]:
# This is a manually collected dictionary of incorrect/faulty keywords in scraped site data
with open(f'{scraped_data}CBS/Data/keyword_trans_dict.txt') as f:
    keyword_trans_dict = json.load(f)


In [4]:
# 114 words to fix
len(keyword_trans_dict)


114

In [5]:
def fix_broken_linkedin_files(glob_path):
    fix_list = []
    data_dict = {}
    data_list = []

    if glob_path.endswith('.json'):

        with open(glob_path, encoding = 'utf-8') as csv_file_handler:
            csv_reader = csv.DictReader(csv_file_handler)

            for rows in csv_reader:
                first_key = str(list(rows.keys())[0])
                key = rows[first_key]
                data_dict[key] = rows

        for num in data_dict:
            data_list.append(data_dict[num])

        with open(glob_path, 'w', encoding = 'utf-8') as json_file_handler:
            json_file_handler.write(json.dumps(data_list, indent = 4))

    return data_list


In [6]:
def fix_keywords(df_temp):

    # This is a manually collected dictionary of incorrect/faulty keywords in scraped site data
    with open(f'{scraped_data}CBS/Data/keyword_trans_dict.txt') as f:
        keyword_trans_dict = json.load(f)

    if len(df_temp) > 0 and isinstance(df_temp, pd.DataFrame):
        for key, value in keyword_trans_dict.items():
            df_temp.loc[
                df_temp[df_temp['Search Keyword'].notnull()]['Search Keyword'].astype(str).progress_apply(
                lambda x: x.lower().strip()
                ) == str(key).lower().strip(), 'Search Keyword'
            ] = str(value).lower().strip()

        unfixed = df_temp.loc[
            df_temp[df_temp['Search Keyword'].notnull()]['Search Keyword'].astype(str).progress_apply(lambda x: x.lower().strip()).isin([x.lower().strip() for x in list(keyword_trans_dict.keys())])
        ]

        if len(unfixed) != 0:
            for key, value in keyword_trans_dict.items():
                for idx, row in df_temp.iterrows():
                    if row['Search Keyword'].astype(str).lower().strip() == str(key).lower().strip():
                        df_temp.loc[idx, 'Search Keyword'] = str(value).lower().strip()

        unfixed = df_temp.loc[
                df_temp[df_temp['Search Keyword'].notnull()]['Search Keyword'].astype(str).progress_apply(lambda x: x.lower().strip()).isin([x.lower().strip() for x in list(keyword_trans_dict.keys())])
            ]
        if len(unfixed) != 0:
            print('Some keywords were not fixed. Please check file unfixed_keywords.txt in data directory.')
            with open(f'{data_dir}unfixed_keywords.txt', 'w') as f:
                json.dump(unfixed, f)

    return df_temp


#### Read paths

In [7]:
glob_paths = []

for site in site_list:
    glob_paths.extend(glob.glob(f'{scraped_data}/{site}/Data/*.json')+glob.glob(f'{scraped_data}/{site}/Data/*.csv'))


In [8]:
# 955 json and csv files
len(glob_paths)


955

#### Use paths to open files, fix keywords, and drop unneeded columns

In [9]:
%%time
# Fix list catches all incorrect/faculty keyword search terms
fix_list = []

# Appended data catches all the fixed and cleaned dfs
appended_data = []

for glob_path in glob_paths:

    if glob_path.endswith('.json'):
        try:
            df_temp = pd.read_json(glob_path)
        except ValueError:
            fix_list.append(glob_path)
            if '1. Scraping/LinkedIn/Data/linkedin_jobs_df_' in glob_path:
                data_json = fix_broken_linkedin_files(glob_path)
                try:
                    df_temp = pd.read_json(glob_path)
                except ValueError:
                    fix_list.append(glob_path)
    elif glob_path.endswith('.csv'):
        df_temp = pd.read_csv(glob_path)
    if len(df_temp) > 0 and isinstance(df_temp, pd.DataFrame):
        df_temp = fix_keywords(df_temp)
        df_temp = df_temp
        # df_temp = df_temp.drop(columns=cols, axis='columns', errors='ignore')
        df_temp = df_temp.drop(
        df_temp.columns[
                df_temp.columns.str.contains(
                    'unnamed|index|level', regex=True, case=False, flags=re.I
                )
            ],
            axis='columns',
            errors='ignore',
        )

        if glob_path.endswith('.json'):
            df_temp.to_json(glob_path, orient='records')
        elif glob_path.endswith('.csv'):
            df_temp.to_csv(glob_path, index=False)

        appended_data.append(df_temp)

# Concatonate list of dfs into one large df_jobs
df_jobs = pd.concat(appended_data, axis='index')

# Save df_jobs to file
if len(df_jobs) > 0 and isinstance(df_jobs, pd.DataFrame):
    df_jobs.to_pickle(f'{df_save_dir}df_raw.pkl')
    df_jobs.to_csv(f'{df_save_dir}df_raw.csv', index=False)
else:
    print(f'ERORR: LENGTH OF DF = {len(df_jobs)}')


CPU times: user 41.7 s, sys: 1.78 s, total: 43.4 s
Wall time: 45.1 s


In [10]:
# If we couldn't fix some keywords, we add them to list fix_list and write to file
if len(fix_list) != 0:
    print('Some keywords to fix!')
    with open(f'{data_dir}fix_list.txt', 'w') as f:
        json.dump(fix_list, f)


In [11]:
# List of dfs, len = 527
len(appended_data)


527

In [12]:
# Concatonate list of dfs into one large df_jobs
df_jobs = pd.concat(appended_data)


In [13]:
# len = 204113
len(df_jobs)


204113

In [14]:
df_jobs.columns


Index(['Search Keyword', 'Platform', 'Job ID', 'Job Title', 'Company Name', 'Location', 'Job Description', 'Rating', 'Employment Type', 'Company URL', 'Job URL', 'Job Age', 'Job Age Number', 'Collection Date', 'Data Row', 'Tracking ID', 'Industry', 'Job Date', 'Type of ownership'], dtype='object')

In [15]:
# Append keywords to list and save to file
keywords = list(set(df_jobs['Search Keyword'].tolist()))
print(f'Number of unique keywords: {len(keywords)}')
with open(f'{data_dir}unique_search_keywords_len({len(keywords)}).txt', 'w') as f:
    json.dump(keywords, f)


Number of unique keywords: 80


In [16]:
keywords[:10]


['quarrying',
 'production',
 'service activity',
 'recreation',
 'engineer',
 'food serving',
 'logistics manager',
 'social work activity',
 'communication',
 'energy']

In [17]:
# Save df_jobs to file
assert len(df_jobs) > 0 and isinstance(df_jobs, pd.DataFrame), f'ERORR: LENGTH OF DF = {len(df_jobs)}'
df_jobs.to_pickle(f'{df_save_dir}df_raw.pkl')
df_jobs.to_csv(f'{df_save_dir}df_raw.csv', index=False)


# Drop duplicated and missing data

### START HERE IF SOURCING FROM DF_JOBS_RAW
### PLEASE SET CORRECT DIRECTORY PATHS BELOW

In [18]:
import os # type:ignore # isort:skip # fmt:skip # noqa # nopep8
import sys # type:ignore # isort:skip # fmt:skip # noqa # nopep8
from pathlib import Path # type:ignore # isort:skip # fmt:skip # noqa # nopep8

mod = sys.modules[__name__]

code_dir = None
code_dir_name = 'Code'
unwanted_subdir_name = 'Analysis'

if code_dir_name not in str(Path.cwd()).split('/')[-1]:
    for _ in range(5):

        parent_path = str(Path.cwd().parents[_]).split('/')[-1]

        if (code_dir_name in parent_path) and (unwanted_subdir_name not in parent_path):

            code_dir = str(Path.cwd().parents[_])

            if code_dir is not None:
                break
else:
    code_dir = str(Path.cwd())
sys.path.append(code_dir)

# %load_ext autoreload
# %autoreload 2


In [19]:
from setup_module.imports import * # type:ignore # isort:skip # fmt:skip # noqa # nopep8


In [20]:
df_jobs = pd.read_pickle(f'{df_save_dir}df_raw.pkl')


In [ ]:
df_jobs['Job Description'] = df_jobs['Job Description'].progress_apply(lambda x: unicodedata.normalize('NFKD', x.encode('ascii', 'ignore').decode('utf-8', 'ignore')))


In [21]:
# len = 204113
len(df_jobs)


204113

In [22]:
df_jobs.info()


<class 'pandas.core.frame.DataFrame'>
RangeIndex: 204113 entries, 0 to 204112
Data columns (total 19 columns):
 #   Column             Non-Null Count   Dtype  
---  ------             --------------   -----  
 0   Search Keyword     204113 non-null  object 
 1   Platform           204113 non-null  object 
 2   Job ID             204113 non-null  object 
 3   Job Title          204113 non-null  object 
 4   Company Name       204103 non-null  object 
 5   Location           204113 non-null  object 
 6   Job Description    204098 non-null  object 
 7   Rating             51158 non-null   float64
 8   Employment Type    203053 non-null  object 
 9   Company URL        193659 non-null  object 
 10  Job URL            204113 non-null  object 
 11  Job Age            204113 non-null  object 
 12  Job Age Number     204113 non-null  object 
 13  Collection Date    204113 non-null  object 
 14  Data Row           152949 non-null  float64
 15  Tracking ID        152949 non-null  object 
 16  In

In [23]:
df_jobs.columns


Index(['Search Keyword', 'Platform', 'Job ID', 'Job Title', 'Company Name', 'Location', 'Job Description', 'Rating', 'Employment Type', 'Company URL', 'Job URL', 'Job Age', 'Job Age Number', 'Collection Date', 'Data Row', 'Tracking ID', 'Industry', 'Job Date', 'Type of ownership'], dtype='object')

In [24]:
# Clean columns
df_jobs.columns = df_jobs.columns.to_series().progress_apply(lambda x: str(x).strip())


In [25]:
# Missing values: Job Description = 15, Job ID = 0
df_jobs.isna().sum()


Search Keyword            0
Platform                  0
Job ID                    0
Job Title                 0
Company Name             10
Location                  0
Job Description          15
Rating               152955
Employment Type        1060
Company URL           10454
Job URL                   0
Job Age                   0
Job Age Number            0
Collection Date           0
Data Row              51164
Tracking ID           51164
Industry              50098
Job Date              51158
Type of ownership    203053
dtype: int64

In [26]:
# Drop NA
df_jobs = df_jobs.dropna(axis='index', how='all')
df_jobs = df_jobs.dropna(axis='columns', how='all')
df_jobs['Job Description'] = df_jobs['Job Description'].progress_apply(lambda x: unicodedata.normalize('NFKD', x.encode('ascii', 'ignore').decode('utf-8', 'ignore')))


In [28]:
# len = 204085
len(df_jobs)


204085

In [ ]:
df_jobs['Job Description'] = df_jobs['Job Description'].astype(str)


In [ ]:
df_jobs['Job Description'] = df_jobs['Job Description'].progress_apply(lambda x: x.strip())


In [ ]:
df_jobs['Job Description'] = df_jobs[
    'Job Description'
].progress_apply(lambda x: x if isinstance(x, str) else ast.literal_eval(x))
df_jobs['Job Description'] = df_jobs['Job Description'].progress_apply(lambda x: unicodedata.normalize('NFKD', x.encode('ascii', 'ignore').decode('utf-8', 'ignore')))


In [29]:
# Drop duplicates on subset of 'Job Description'
df_jobs = df_jobs.drop_duplicates(subset=['Job Description'], keep='first')


In [30]:
# Conver Job ID and Sentence to str
str_cols = [
    'Job ID',
    'Job Description',
]

for col in str_cols:
    df_jobs[col] = df_jobs[col].astype(str)
    print(f'{col} converted to str.' if all(df_jobs[col].progress_apply(lambda x: isinstance(x, str))) else f'{col} NOT converted to str.')


Job ID converted to str.
Job Description converted to str.


In [31]:
# Drop duplicates on subset of 'Job Description'
df_jobs = df_jobs.drop_duplicates(subset=['Job ID', 'Job Description'], keep='first', ignore_index=True)


In [32]:
# len = 62579
len(df_jobs)


62577

In [34]:
# len = 62577
len(df_jobs)


62577

In [35]:
# Save df_jobs to file
assert len(df_jobs) > 0 and isinstance(df_jobs, pd.DataFrame), f'ERORR: LENGTH OF DF = {len(df_jobs)}'
df_jobs.to_pickle(f'{df_save_dir}df_jobs_raw_dropped.pkl')
df_jobs.to_csv(f'{df_save_dir}df_jobs_raw_dropped.csv', index=False)


# Detect job description language

### START HERE IF SOURCING FROM DF_JOBS_RAW_DROPPED
### PLEASE SET CORRECT DIRECTORY PATHS BELOW


In [3]:
import os # type:ignore # isort:skip # fmt:skip # noqa # nopep8
import sys # type:ignore # isort:skip # fmt:skip # noqa # nopep8
from pathlib import Path # type:ignore # isort:skip # fmt:skip # noqa # nopep8

mod = sys.modules[__name__]

code_dir = None
code_dir_name = 'Code'
unwanted_subdir_name = 'Analysis'

if code_dir_name not in str(Path.cwd()).split('/')[-1]:
    for _ in range(5):

        parent_path = str(Path.cwd().parents[_]).split('/')[-1]

        if (code_dir_name in parent_path) and (unwanted_subdir_name not in parent_path):

            code_dir = str(Path.cwd().parents[_])

            if code_dir is not None:
                break
else:
    code_dir = str(Path.cwd())
sys.path.append(code_dir)

# %load_ext autoreload
# %autoreload 2


In [ ]:
from setup_module.imports import *  # type:ignore # isort:skip # fmt:skip # noqa # nopep8


In [6]:
df_jobs = pd.read_pickle(f'{df_save_dir}df_raw_dropped.pkl')


In [7]:
# 62577
len(df_jobs)


62577

In [8]:
df_jobs.info()


<class 'pandas.core.frame.DataFrame'>
RangeIndex: 62577 entries, 0 to 62576
Data columns (total 19 columns):
 #   Column             Non-Null Count  Dtype  
---  ------             --------------  -----  
 0   Search Keyword     62577 non-null  object 
 1   Platform           62577 non-null  object 
 2   Job ID             62577 non-null  object 
 3   Job Title          62577 non-null  object 
 4   Company Name       62574 non-null  object 
 5   Location           62577 non-null  object 
 6   Job Description    62577 non-null  object 
 7   Rating             3975 non-null   float64
 8   Employment Type    61995 non-null  object 
 9   Company URL        59263 non-null  object 
 10  Job URL            62577 non-null  object 
 11  Job Age            62577 non-null  object 
 12  Job Age Number     62577 non-null  object 
 13  Collection Date    62577 non-null  object 
 14  Data Row           58599 non-null  float64
 15  Tracking ID        58599 non-null  object 
 16  Industry           591

In [10]:
# This is a manually collected dictionary of incorrect/faulty keywords in scraped site data
with open(f'{scraped_data}CBS/Data/keyword_trans_dict.txt') as f:
    keyword_trans_dict = json.load(f)

if len(df_jobs['Search Keyword'].loc[df_jobs['Search Keyword'].isin(list(keyword_trans_dict.keys()))]) != 0:
    df_jobs = fix_keywords(df_jobs)


In [14]:
%%time
translator = Translator()
googletrans_readtime_error = googletrans.client.httpx._client.httpcore._exceptions.ReadTimeout

if 'Language' not in df_jobs.columns:
    df_jobs['Language'] = np.nan # create Language col and fill it with nan

# try:
#     time.sleep(60)
#     df_jobs['Language'] = df_jobs['Job Description'].progress_apply(lambda x: translator.detect(str(x).lower().strip()).lang)
# except:
#     time.sleep(3600)
#     df_jobs['Language'] = df_jobs['Job Description'].progress_apply(lambda x: translator.detect(str(x).lower().strip()).lang)

for idx, row in df_jobs.iterrows():
    # This part ensures we don't start lang detection from index 0 if some lang detection was already done
    if len(str(row['Job Description'])) != 0:
        if type(row['Language']) == float and np.isnan(row['Language']): #if lang is nan, detect language

            try:
                print(f'Row {idx}: Language detection in progress.')
#                 time.sleep(10)
                df_jobs.loc[idx, 'Language'] = str(translator.detect(str(row['Job Description']).lower().strip()).lang)
                print(f'Row {idx}: Language detection done.')
            except:# googletrans_readtime_error:

                if len(df_jobs) > 0 and isinstance(df_jobs, pd.DataFrame):
                    df_jobs.to_pickle(f'{df_save_dir}df_raw_language_detected.pkl')

                    df_jobs.to_csv(f'{df_save_dir}df_raw_language_detected.csv', index=False)
                else:
                    print(f'ERORR: LENGTH OF DF = {len(df_jobs)}')

                print(f'Row {idx}: Sleeping for half an hour starting at {time.strftime("%I:%M:%S %p", time.localtime())}.')
                print('-'*30)
                time.sleep(1800)
                print(f'Row {idx}: Done sleeping.')
                print('-'*30)
                print(f'Row {idx}: Language detection in progress.')
                df_jobs.loc[idx, 'Language'] = str(translator.detect(str(row['Job Description']).lower().strip()).lang)
                print(f'Row {idx}: Language detection done.')

        else: # elif lang is not nan, skip and go to next idx
            continue

if len(df_jobs) > 0 and isinstance(df_jobs, pd.DataFrame):
    df_jobs.to_pickle(f'{df_save_dir}df_raw_language_detected.pkl')

    df_jobs.to_csv(f'{df_save_dir}df_raw_language_detected.csv', index=False)
else:
    print(f'ERORR: LENGTH OF DF = {len(df_jobs)}')


In [15]:
# nl = 44863, en = 17591, ['en', 'nl'] = 8
df_jobs['Language'].value_counts()


nl              44863
en              17591
de                 53
fr                 36
['nl', 'en']        9
['en', 'nl']        8
pl                  5
id                  4
da                  4
tr                  1
['nl', 'af']        1
st                  1
af                  1
Name: Language, dtype: int64

In [ ]:
assert len(df_jobs) > 0 and isinstance(df_jobs, pd.DataFrame), f'ERORR: LENGTH OF DF = {len(df_jobs)}'
df_jobs.to_pickle(f'{df_save_dir}df_jobs_raw_language_detected.pkl')
df_jobs.to_csv(f'{df_save_dir}df_jobs_raw_language_detected.csv', index=False)
